# Notebook to investigate the reproject_scene.py script

Can use the conda environment `landsat2nc_env2` for running the original code. MAGEO has all of the required libraries we need so using default env for now.

## Running the original code

In [ ]:
# imports for running the original code:
import xarray as xr
from osgeo import osr, gdal
import numpy as np
import logging

In [ ]:
# Projection class from original code. Will leave this unchanged for now
# as it is just used once at the start of the processing to get the
# projection info from the input file and define the target projection.

class Projection:

    wgs84_wkt = """
        GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.01745329251994328,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]]"""

    def __init__(self,source_projection):
        self.source_projeciton = source_projection

        # WGS coordinate system
        wgs84_coord_sys = osr.SpatialReference()
        wgs84_coord_sys.ImportFromWkt(Projection.wgs84_wkt)

        srs = osr.SpatialReference()
        srs.ImportFromWkt(self.source_projeciton)

        self.source_to_wgs84 = osr.CoordinateTransformation(srs, wgs84_coord_sys)

    def get_source_to_wgs84_transform(self):
        return self.source_to_wgs84

    @staticmethod
    def setup(source_projection):
        Projection.projector = Projection(source_projection)

In [ ]:
# Original TiffImporter class. This contains the function to reproject the
# image and is the bit that takes time. The code below will be modified to
# see if it can run faster on a GPU.

class TiffImporter:

    def __init__(self):
        pass

    def latlon_image(self, path):

        # Open input dataset
        indataset = gdal.Open(path, gdal.GA_ReadOnly)
        Projection.setup(indataset.GetProjection())

        # Read geotransform matrix and calculate ground coordinates
        geomatrix = indataset.GetGeoTransform()
        pixel = indataset.RasterXSize
        line = indataset.RasterYSize

        ct = Projection.projector.get_source_to_wgs84_transform()

        latlon_im = np.zeros([2, line, pixel])

        logging.getLogger().info("Mapping pixel locations to target projection")
        pct = -1
        for i in np.arange(0, line, 1):
            new_pct = int(100*i/line)
            if new_pct > pct and new_pct % 5 == 0:
                logging.getLogger().info("%d"%new_pct + "%")
                pct = new_pct

            for j in np.arange(0, pixel, 1):

                # step 1 - apply geotransform to get the UTM coordinates of the pixel at i,j
                X = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
                Y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i

                # Shift to the center of the pixel
                X += geomatrix[1] / 2.0
                Y += geomatrix[5] / 2.0

                # step 2 - apply reprojection from UTM to WGS84
                (lat,lon,_) = ct.TransformPoint(X, Y)

                latlon_im[0, i, j] = lat
                latlon_im[1, i, j] = lon

        return latlon_im

In [ ]:
# Need to download the test data:
#!wget https://gws-access.jasmin.ac.uk/public/nceo_uor/niall/LC08_L2SP_201024_20180629_20200831_02_T1/LC08_L2SP_201024_20180629_20200831_02_T1_ST_B10.TIF -P data/

In [ ]:
# The next few cells contain the main code in the reproject_scene.py script
# that runs on the example file
input_path =  'data/LC08_L2SP_201024_20180629_20200831_02_T1_ST_B10.TIF'   # path to a Landsat8 TIF file to read
output_path = 'coordinates.nc'   # path to output netcdf4 file containing pixel lat/lon coordinates
#logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
#logging.info("Extracting WGS84 pixel coordinates from image %s to %s" % (input_path, output_path))
importer = TiffImporter()

In [ ]:
%%time
# the bit that takes the time:
latlon_im = importer.latlon_image(input_path)

In [ ]:
logging.info("Extracted coordinates - shape %s" % str(latlon_im.shape))
da = xr.DataArray(data=latlon_im, dims=("axis","i","j"),name="latlons")
da.to_netcdf(output_path)
logging.info("Extraction complete")

In [ ]:
# Ran a quick test on the above code by commenting out the step 2 code and checking
# processing time. This took over 8mins to run so step 1 is taking up the bulk
# of the processing time. Speeding up this step alone will make a big difference to overall
# time.

## Vectorize step 1

In [ ]:
from numba import vectorize, cuda

In [ ]:
# device = cuda.get_current_device()
# device.reset()

In [ ]:
# cuda.select_device(0)
# cuda.close()
# cuda.select_device(0)

In [ ]:
# run the first few bits of the latlon_image function to use for testing vectorized
# functions...

# Open input dataset
indataset = gdal.Open(input_path, gdal.GA_ReadOnly)
Projection.setup(indataset.GetProjection())

# Read geotransform matrix and calculate ground coordinates
geomatrix = indataset.GetGeoTransform()
pixel = indataset.RasterXSize
line = indataset.RasterYSize

ct = Projection.projector.get_source_to_wgs84_transform()

In [ ]:
# CPU

In [ ]:
# functions to apply geotransform to get the UTM coordinates of the pixel at i,j
# separate functions for x and y - probably a nicer way of handling this!

@vectorize
def geotransform_x_cpu(i, j):
    x = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
    # Shift to the center of the pixel
    x += geomatrix[1] / 2.0
    return x

@vectorize
def geotransform_y_cpu(i, j):
    y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i
    # Shift to the center of the pixel
    y += geomatrix[5] / 2.0
    return y

In [ ]:
i = np.arange(0, line, 1).reshape(line, 1)
j = np.arange(0, pixel, 1).reshape(1, pixel)

In [ ]:
%%timeit
x = geotransform_x_cpu(i, j)
y = geotransform_y_cpu(i, j)

In [ ]:
# a significant improvement! (about 8mins down to half a sec)

In [ ]:
## how about numpy vectorize:
def geotransform_x_cpu(i, j):
    x = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
    # Shift to the center of the pixel
    x += geomatrix[1] / 2.0
    return x

geo_x_vec = np.vectorize(geotransform_x_cpu)
%timeit geo_x_vec(i, j)

In [ ]:
from numba import jit, vectorize

@jit
def geotransform_x_cpu(i, j):
    x = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
    # Shift to the center of the pixel
    x += geomatrix[1] / 2.0
    return x

@jit
def geotransform_y_cpu(i, j):
    y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i
    # Shift to the center of the pixel
    y += geomatrix[5] / 2.0
    return y


In [ ]:
%%timeit
geotransform_x_cpu(i, j)
geotransform_y_cpu(i, j)

In [ ]:
def geotransform_x_cpu(i, j):
    x = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
    # Shift to the center of the pixel
    x += geomatrix[1] / 2.0
    return x

def geotransform_y_cpu(i, j):
    y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i
    # Shift to the center of the pixel
    y += geomatrix[5] / 2.0
    return y


In [ ]:
%%timeit
# just np...
geotransform_x_cpu(i, j)
geotransform_y_cpu(i, j)

In [ ]:
# fastest method is with numba vectorize

In [ ]:
# GPU

In [ ]:
@vectorize(['float64(int64, int64)'], target='cuda')
def geotransform_x_gpu(i, j):
    x = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
    x += geomatrix[1] / 2.0
    return x

@vectorize(['float64(int64, int64)'], target='cuda')
def geotransform_y_gpu(i, j):
    y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i
    y += geomatrix[5] / 2.0
    return y

In [ ]:
%%time
geotransform_x_gpu(i, j)
geotransform_y_gpu(i, j)

In [ ]:
# this takes over 1 sec, so slower than using CPU - probably due to copying data to/from device, check...

In [ ]:
# del d_x, d_y, d_i, d_j

In [ ]:
%%time
d_i = cuda.to_device(i) #input
d_j = cuda.to_device(j) #input
d_x = cuda.device_array(shape=(8081, 8181), dtype=np.float64) #output
d_y = cuda.device_array(shape=(8081, 8181), dtype=np.float64) #output

geotransform_x_gpu(d_i, d_j, out=d_x)
geotransform_y_gpu(d_i, d_j, out=d_y)

x = d_x.copy_to_host()
y = d_y.copy_to_host()

In [ ]:
d_i = cuda.to_device(i)
d_j = cuda.to_device(j)
d_x = cuda.device_array(shape=(8081, 8181), dtype=np.float64)
d_y = cuda.device_array(shape=(8081, 8181), dtype=np.float64)

In [ ]:
%%timeit
# time the processing only:
geotransform_x_gpu(d_i, d_j, out=d_x)
geotransform_y_gpu(d_i, d_j, out=d_y)

In [ ]:
x = d_x.copy_to_host()
y = d_y.copy_to_host()

In [ ]:
# time for processing alone is much less. So although CPU may be faster for step 1 alone,
# may be overall faster when perform both steps on GPU...
# NOTE: above I am still seeing longer timings for GPU than CPU - this had gone down to a few microseconds
# in the original notebook so I'm not sure why slower today.
# The first time you run it, the function will take longer because it has to compile. Running again you get much faster
# timings. To see fastest results would be good to run the script on multiple scenes in same script to make use of cache.

In [ ]:
# Try plugging in step 1 into the TiffImporter class to ensure get same results and look at new timings...

In [ ]:
# Adding the functions within class for now because they need the local variables (geomatrix)
class TiffImporter:

    def __init__(self):
        pass

    def latlon_image(self, path):

        # Open input dataset
        indataset = gdal.Open(path, gdal.GA_ReadOnly)
        Projection.setup(indataset.GetProjection())

        # Read geotransform matrix and calculate ground coordinates
        geomatrix = indataset.GetGeoTransform()
        pixel = indataset.RasterXSize
        line = indataset.RasterYSize

        ct = Projection.projector.get_source_to_wgs84_transform()

        latlon_im = np.zeros([2, line, pixel])

        logging.getLogger().info("Mapping pixel locations to target projection")

        # step 1 - apply geotransform to get the UTM coordinates of the pixel at i,j
        @vectorize(['float64(int64, int64)'], target='cuda')
        def geotransform_x_gpu(i, j):
            x = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
            x += geomatrix[1] / 2.0
            return x

        @vectorize(['float64(int64, int64)'], target='cuda')
        def geotransform_y_gpu(i, j):
            y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i
            y += geomatrix[5] / 2.0
            return y
        
        i = np.arange(0, line, 1).reshape(line, 1)
        j = np.arange(0, pixel, 1).reshape(1, pixel)
        
        d_i = cuda.to_device(i)
        d_j = cuda.to_device(j)
        d_x = cuda.device_array(shape=(line, pixel), dtype=np.float64)
        d_y = cuda.device_array(shape=(line, pixel), dtype=np.float64)

        geotransform_x_gpu(d_i, d_j, out=d_x)
        geotransform_y_gpu(d_i, d_j, out=d_y)

        x = d_x.copy_to_host()
        y = d_y.copy_to_host()
        
        for i in np.arange(0, line, 1):
            for j in np.arange(0, pixel, 1):
                X = x[i, j]
                Y = y[i, j]
                
                # step 2 - apply reprojection from UTM to WGS84
                (lat,lon,_) = ct.TransformPoint(X, Y)

                latlon_im[0, i, j] = lat
                latlon_im[1, i, j] = lon

        return latlon_im

In [ ]:
%%time
importer = TiffImporter()
latlon_im = importer.latlon_image(input_path)

In [ ]:
latlon_im.shape

In [ ]:
da_gpu = xr.DataArray(data=latlon_im, dims=("axis","i","j"),name="latlons")

In [ ]:
da_orig = xr.open_dataarray(output_path)

In [ ]:
np.all(da_gpu == da_orig)

In [ ]:
# so yes it gives same result and we've gone from 12mins down to 3mins

## Vectorize step 2

In [ ]:
# step 2 - apply reprojection from UTM to WGS84
# what is happening here: (lat,lon,_) = ct.TransformPoint(X, Y)
# and how can we port it to GPU?

In [ ]:
# (Need to look more in to what PROJ does for the conversion.)

In [ ]:
# found an alternative example where it does the conversion (possibly less accurate?) with pure python:
import math as mathlib

__all__ = ['to_latlon', 'from_latlon']

K0 = 0.9996

E = 0.00669438
E2 = E * E
E3 = E2 * E
E_P2 = E / (1.0 - E)

SQRT_E = mathlib.sqrt(1 - E)
_E = (1 - SQRT_E) / (1 + SQRT_E)
_E2 = _E * _E
_E3 = _E2 * _E
_E4 = _E3 * _E
_E5 = _E4 * _E

M1 = (1 - E / 4 - 3 * E2 / 64 - 5 * E3 / 256)
M2 = (3 * E / 8 + 3 * E2 / 32 + 45 * E3 / 1024)
M3 = (15 * E2 / 256 + 45 * E3 / 1024)
M4 = (35 * E3 / 3072)

P2 = (3. / 2 * _E - 27. / 32 * _E3 + 269. / 512 * _E5)
P3 = (21. / 16 * _E2 - 55. / 32 * _E4)
P4 = (151. / 96 * _E3 - 417. / 128 * _E5)
P5 = (1097. / 512 * _E4)

R = 6378137

ZONE_LETTERS = "CDEFGHJKLMNPQRSTUVWXX"

def in_bounds(x, lower, upper, upper_strict=False):
    if upper_strict and use_numpy:
        return lower <= mathlib.min(x) and mathlib.max(x) < upper
    elif upper_strict and not use_numpy:
        return lower <= x < upper
    elif use_numpy:
        return lower <= mathlib.min(x) and mathlib.max(x) <= upper
    return lower <= x <= upper

def mixed_signs(x):
    return use_numpy and mathlib.min(x) < 0 and mathlib.max(x) >= 0

def negative(x):
    if use_numpy:
        return mathlib.max(x) < 0
    return x < 0

def to_latlon(easting, northing, zone_number, zone_letter=None, northern=None, strict=True):

    if not zone_letter and northern is None:
        raise ValueError('either zone_letter or northern needs to be set')

    elif zone_letter and northern is not None:
        raise ValueError('set either zone_letter or northern, but not both')

    if strict:
        if not in_bounds(easting, 100000, 1000000, upper_strict=True):
            raise OutOfRangeError('easting out of range (must be between 100.000 m and 999.999 m)')
        if not in_bounds(northing, 0, 10000000):
            raise OutOfRangeError('northing out of range (must be between 0 m and 10.000.000 m)')

    if zone_letter:
        zone_letter = zone_letter.upper()
        northern = (zone_letter >= 'N')

    x = easting - 500000
    y = northing

    if not northern:
        y -= 10000000

    m = y / K0
    mu = m / (R * M1)

    p_rad = (mu +
             P2 * mathlib.sin(2 * mu) +
             P3 * mathlib.sin(4 * mu) +
             P4 * mathlib.sin(6 * mu) +
             P5 * mathlib.sin(8 * mu))

    p_sin = mathlib.sin(p_rad)
    p_sin2 = p_sin * p_sin

    p_cos = mathlib.cos(p_rad)
    
    p_tan = p_sin / p_cos
    p_tan2 = p_tan * p_tan
    p_tan4 = p_tan2 * p_tan2

    ep_sin = 1 - E * p_sin2
    ep_sin_sqrt = mathlib.sqrt(1 - E * p_sin2)

    n = R / ep_sin_sqrt
    r = (1 - E) / ep_sin

    c = _E * p_cos**2
    c2 = c * c

    d = x / (n * K0)
    d2 = d * d
    d3 = d2 * d
    d4 = d3 * d
    d5 = d4 * d
    d6 = d5 * d

    latitude = (p_rad - (p_tan / r) *
                (d2 / 2 -
                 d4 / 24 * (5 + 3 * p_tan2 + 10 * c - 4 * c2 - 9 * E_P2)) +
                 d6 / 720 * (61 + 90 * p_tan2 + 298 * c + 45 * p_tan4 - 252 * E_P2 - 3 * c2))

    longitude = (d -
                 d3 / 6 * (1 + 2 * p_tan2 + c) +
                 d5 / 120 * (5 - 2 * c + 28 * p_tan2 - 3 * c2 + 8 * E_P2 + 24 * p_tan4)) / p_cos

    return (mathlib.degrees(latitude),
            mathlib.degrees(longitude) + zone_number_to_central_longitude(zone_number))

def from_latlon(latitude, longitude, force_zone_number=None, force_zone_letter=None):

    if not in_bounds(latitude, -80.0, 84.0):
        raise OutOfRangeError('latitude out of range (must be between 80 deg S and 84 deg N)')
    if not in_bounds(longitude, -180.0, 180.0):
        raise OutOfRangeError('longitude out of range (must be between 180 deg W and 180 deg E)')

    lat_rad = mathlib.radians(latitude)
    lat_sin = mathlib.sin(lat_rad)
    lat_cos = mathlib.cos(lat_rad)

    lat_tan = lat_sin / lat_cos
    lat_tan2 = lat_tan * lat_tan
    lat_tan4 = lat_tan2 * lat_tan2

    if force_zone_number is None:
        zone_number = latlon_to_zone_number(latitude, longitude)
    else:
        zone_number = force_zone_number

    if force_zone_letter is None:
        zone_letter = latitude_to_zone_letter(latitude)
    else:
        zone_letter = force_zone_letter

    lon_rad = mathlib.radians(longitude)
    central_lon = zone_number_to_central_longitude(zone_number)
    central_lon_rad = mathlib.radians(central_lon)

    n = R / mathlib.sqrt(1 - E * lat_sin**2)
    c = E_P2 * lat_cos**2

    a = lat_cos * (lon_rad - central_lon_rad)
    a2 = a * a
    a3 = a2 * a
    a4 = a3 * a
    a5 = a4 * a
    a6 = a5 * a

    m = R * (M1 * lat_rad -
             M2 * mathlib.sin(2 * lat_rad) +
             M3 * mathlib.sin(4 * lat_rad) -
             M4 * mathlib.sin(6 * lat_rad))

    easting = K0 * n * (a +
                        a3 / 6 * (1 - lat_tan2 + c) +
                        a5 / 120 * (5 - 18 * lat_tan2 + lat_tan4 + 72 * c - 58 * E_P2)) + 500000

    northing = K0 * (m + n * lat_tan * (a2 / 2 +
                                        a4 / 24 * (5 - lat_tan2 + 9 * c + 4 * c**2) +
                                        a6 / 720 * (61 - 58 * lat_tan2 + lat_tan4 + 600 * c - 330 * E_P2)))

    if mixed_signs(latitude):
        raise ValueError("latitudes must all have the same sign")
    elif negative(latitude):
        northing += 10000000

    return easting, northing, zone_number, zone_letter

def latitude_to_zone_letter(latitude):
    # If the input is a numpy array, just use the first element
    # User responsibility to make sure that all points are in one zone
    if use_numpy and isinstance(latitude, mathlib.ndarray):
        latitude = latitude.flat[0]

    if -80 <= latitude <= 84:
        return ZONE_LETTERS[int(latitude + 80) >> 3]
    else:
        return None
    
def latlon_to_zone_number(latitude, longitude):
    # If the input is a numpy array, just use the first element
    # User responsibility to make sure that all points are in one zone
    if use_numpy:
        if isinstance(latitude, mathlib.ndarray):
            latitude = latitude.flat[0]
        if isinstance(longitude, mathlib.ndarray):
            longitude = longitude.flat[0]

    if 56 <= latitude < 64 and 3 <= longitude < 12:
        return 32

    if 72 <= latitude <= 84 and longitude >= 0:
        if longitude < 9:
            return 31
        elif longitude < 21:
            return 33
        elif longitude < 33:
            return 35
        elif longitude < 42:
            return 37

    return int((longitude + 180) / 6) + 1

def zone_number_to_central_longitude(zone_number):
    return (zone_number - 1) * 6 - 180 + 3

In [ ]:
# the to_latlon function is the one we need to use.
# need to make a few tweaks to use with Numba.

In [ ]:
@cuda.jit(device=True)
def to_latlon(easting, northing, zone_number, zone_letter, northern, strict):
    x = easting - 500000
    y = northing

    if not northern:
        y -= 10000000

    m = y / K0
    mu = m / (R * M1)

    p_rad = (mu +
             P2 * mathlib.sin(2 * mu) +
             P3 * mathlib.sin(4 * mu) +
             P4 * mathlib.sin(6 * mu) +
             P5 * mathlib.sin(8 * mu))

    p_sin = mathlib.sin(p_rad)
    p_sin2 = p_sin * p_sin

    p_cos = mathlib.cos(p_rad)

    p_tan = p_sin / p_cos
    p_tan2 = p_tan * p_tan
    p_tan4 = p_tan2 * p_tan2

    ep_sin = 1 - E * p_sin2
    ep_sin_sqrt = mathlib.sqrt(1 - E * p_sin2)

    n = R / ep_sin_sqrt
    r = (1 - E) / ep_sin

    c = _E * p_cos**2
    c2 = c * c

    d = x / (n * K0)
    d2 = d * d
    d3 = d2 * d
    d4 = d3 * d
    d5 = d4 * d
    d6 = d5 * d

    latitude = (p_rad - (p_tan / r) *
                (d2 / 2 -
                 d4 / 24 * (5 + 3 * p_tan2 + 10 * c - 4 * c2 - 9 * E_P2)) +
                 d6 / 720 * (61 + 90 * p_tan2 + 298 * c + 45 * p_tan4 - 252 * E_P2 - 3 * c2))

    longitude = (d -
                 d3 / 6 * (1 + 2 * p_tan2 + c) +
                 d5 / 120 * (5 - 2 * c + 28 * p_tan2 - 3 * c2 + 8 * E_P2 + 24 * p_tan4)) / p_cos
    
    def zone_number_to_central_longitude(zone_number):
        return (zone_number - 1) * 6 - 180 + 3
    
    return (mathlib.degrees(latitude),
            mathlib.degrees(longitude) + zone_number_to_central_longitude(zone_number))


@vectorize(['float64(float64, float64)'], target='cuda')
def reproj_gpu_lat(x, y):
    lat, lon = to_latlon(x, y, np.int64(31), None, True, None)
    return lat

@vectorize(['float64(float64, float64)'], target='cuda')
def reproj_gpu_lon(x, y):
    lat, lon = to_latlon(x, y, np.int64(31), None, True, None)
    return lon

In [ ]:
print(reproj_gpu_lat(205500.0, 5852100.0)); print(reproj_gpu_lon(205500.0, 5852100.0))

In [ ]:
# compare to reults using proj:
(52.73838067, -1.36309792)

In [ ]:
# plug back into the TiffImporter class:

In [ ]:
class TiffImporter:

    def __init__(self):
        pass

    def latlon_image(self, path):

        # Open input dataset
        indataset = gdal.Open(path, gdal.GA_ReadOnly)
        Projection.setup(indataset.GetProjection())

        # Read geotransform matrix and calculate ground coordinates
        geomatrix = indataset.GetGeoTransform()
        pixel = indataset.RasterXSize
        line = indataset.RasterYSize

        ct = Projection.projector.get_source_to_wgs84_transform()

        latlon_im = np.zeros([2, line, pixel])
        
        # step 1 - apply geotransform to get the UTM coordinates of the pixel at i,j
        @vectorize(['float64(int64, int64)'], target='cuda')
        def transform_x_gpu(i, j):
            x = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
            x += geomatrix[1] / 2.0
            return x

        @vectorize(['float64(int64, int64)'], target='cuda')
        def transform_y_gpu(i, j):
            y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i
            y += geomatrix[5] / 2.0
            return y
        
        i = np.arange(0, line, 1).reshape(line, 1)
        j = np.arange(0, pixel, 1).reshape(1, pixel)
        
        d_i = cuda.to_device(i)
        d_j = cuda.to_device(j)
        d_x = cuda.device_array(shape=(line, pixel), dtype=np.float64)
        d_y = cuda.device_array(shape=(line, pixel), dtype=np.float64)

        transform_x_gpu(d_i, d_j, out=d_x)
        transform_y_gpu(d_i, d_j, out=d_y)
        
        # step 2 - apply reprojection from UTM to WGS84
        d_lat = cuda.device_array(shape=(line, pixel), dtype=np.float64)
        d_lon = cuda.device_array(shape=(line, pixel), dtype=np.float64)
        reproj_gpu_lat(d_x, d_y, out=d_lat)
        reproj_gpu_lon(d_x, d_y, out=d_lon)
        latlon_im[0] = d_lat.copy_to_host()
        latlon_im[1] = d_lon.copy_to_host()

        return latlon_im

In [ ]:
%%time
importer = TiffImporter()
latlon_im = importer.latlon_image(input_path)

In [ ]:
# now down to about 2.5 seconds using the GPU!

In [ ]:
latlon_im - da_gpu

In [ ]:
np.min(latlon_im - da_gpu).data, np.max(latlon_im - da_gpu).data

In [ ]:
# does not give identical results so may want to double check accuracy of the 2 methods?
# how important are these differences in practice?

In [ ]:
# compare to CPU equivalent to see timings...

In [ ]:
from numba import jit, vectorize

@jit
def to_latlon(easting, northing, zone_number, zone_letter, northern, strict):


    x = easting - 500000
    y = northing

    if not northern:
        y -= 10000000

    m = y / K0
    mu = m / (R * M1)

    p_rad = (mu +
             P2 * mathlib.sin(2 * mu) +
             P3 * mathlib.sin(4 * mu) +
             P4 * mathlib.sin(6 * mu) +
             P5 * mathlib.sin(8 * mu))

    p_sin = mathlib.sin(p_rad)
    p_sin2 = p_sin * p_sin

    p_cos = mathlib.cos(p_rad)

    p_tan = p_sin / p_cos
    p_tan2 = p_tan * p_tan
    p_tan4 = p_tan2 * p_tan2

    ep_sin = 1 - E * p_sin2
    ep_sin_sqrt = mathlib.sqrt(1 - E * p_sin2)

    n = R / ep_sin_sqrt
    r = (1 - E) / ep_sin

    c = _E * p_cos**2
    c2 = c * c

    d = x / (n * K0)
    d2 = d * d
    d3 = d2 * d
    d4 = d3 * d
    d5 = d4 * d
    d6 = d5 * d

    latitude = (p_rad - (p_tan / r) *
                (d2 / 2 -
                 d4 / 24 * (5 + 3 * p_tan2 + 10 * c - 4 * c2 - 9 * E_P2)) +
                 d6 / 720 * (61 + 90 * p_tan2 + 298 * c + 45 * p_tan4 - 252 * E_P2 - 3 * c2))

    longitude = (d -
                 d3 / 6 * (1 + 2 * p_tan2 + c) +
                 d5 / 120 * (5 - 2 * c + 28 * p_tan2 - 3 * c2 + 8 * E_P2 + 24 * p_tan4)) / p_cos
    
    def zone_number_to_central_longitude(zone_number):
        return (zone_number - 1) * 6 - 180 + 3
    
    return (mathlib.degrees(latitude),
            mathlib.degrees(longitude) + zone_number_to_central_longitude(zone_number))

@vectorize
def reproj_cpu_lat(x, y):
    lat, lon = to_latlon(x, y, np.int64(31), None, True, None)
    return lat

@vectorize
def reproj_cpu_lon(x, y):
    lat, lon = to_latlon(x, y, np.int64(31), None, True, None)
    return lon

In [ ]:
print(reproj_cpu_lat(205500.0, 5852100.0)); print(reproj_cpu_lon(205500.0, 5852100.0))

In [ ]:
print(reproj_gpu_lat(205500.0, 5852100.0)); print(reproj_gpu_lon(205500.0, 5852100.0))

In [ ]:
class TiffImporter:

    def __init__(self):
        pass

    def latlon_image(self, path):

        # Open input dataset
        indataset = gdal.Open(path, gdal.GA_ReadOnly)
        Projection.setup(indataset.GetProjection())

        # Read geotransform matrix and calculate ground coordinates
        geomatrix = indataset.GetGeoTransform()
        pixel = indataset.RasterXSize
        line = indataset.RasterYSize

        ct = Projection.projector.get_source_to_wgs84_transform()

        latlon_im = np.zeros([2, line, pixel])

        
        # step 1 - apply geotransform to get the UTM coordinates of the pixel at i,j
        @vectorize
        def geotransform_x_cpu(i, j):
            X = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
            # Shift to the center of the pixel
            X += geomatrix[1] / 2.0
            return X

        @vectorize
        def geotransform_y_cpu(i, j):
            Y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i
            # Shift to the center of the pixel
            Y += geomatrix[5] / 2.0
            return Y
        
        i = np.arange(0, line, 1).reshape(line, 1)
        j = np.arange(0, pixel, 1).reshape(1, pixel)
        
        x = geotransform_x_cpu(i, j)
        y = geotransform_y_cpu(i, j)

        # step 2
        lat = reproj_cpu_lat(x, y)
        lon = reproj_cpu_lon(x, y)
        latlon_im[0] = lat
        latlon_im[1] = lon

        return latlon_im

In [ ]:
%%time
importer = TiffImporter()
latlon_im = importer.latlon_image(input_path)

In [ ]:
# this is a good sign - still a massive improvement cutting it down to 17s, but GPU is significantly faster,
# so would save time if running loads of these.

In [ ]:
latlon_im - da_orig

In [ ]:
# If we use CuPy in place of NumPy, Numba can work with the arrays in the same way as the device arrays,
# so not need to do all the manual copying to and from the device.
# We can also copy the final array back at the end rather than in two steps for lat and lon, which should
# speed things up a little bit.
# It is also faster working with float32. Results are unchanged.

In [ ]:
@cuda.jit(device=True)
def to_latlon(easting, northing, zone_number, zone_letter, northern, strict):
    x = easting - 500000
    y = northing

    if not northern:
        y -= 10000000

    m = y / K0
    mu = m / (R * M1)

    p_rad = (mu +
             P2 * mathlib.sin(2 * mu) +
             P3 * mathlib.sin(4 * mu) +
             P4 * mathlib.sin(6 * mu) +
             P5 * mathlib.sin(8 * mu))

    p_sin = mathlib.sin(p_rad)
    p_sin2 = p_sin * p_sin

    p_cos = mathlib.cos(p_rad)

    p_tan = p_sin / p_cos
    p_tan2 = p_tan * p_tan
    p_tan4 = p_tan2 * p_tan2

    ep_sin = 1 - E * p_sin2
    ep_sin_sqrt = mathlib.sqrt(1 - E * p_sin2)

    n = R / ep_sin_sqrt
    r = (1 - E) / ep_sin

    c = _E * p_cos**2
    c2 = c * c

    d = x / (n * K0)
    d2 = d * d
    d3 = d2 * d
    d4 = d3 * d
    d5 = d4 * d
    d6 = d5 * d

    latitude = (p_rad - (p_tan / r) *
                (d2 / 2 -
                 d4 / 24 * (5 + 3 * p_tan2 + 10 * c - 4 * c2 - 9 * E_P2)) +
                 d6 / 720 * (61 + 90 * p_tan2 + 298 * c + 45 * p_tan4 - 252 * E_P2 - 3 * c2))

    longitude = (d -
                 d3 / 6 * (1 + 2 * p_tan2 + c) +
                 d5 / 120 * (5 - 2 * c + 28 * p_tan2 - 3 * c2 + 8 * E_P2 + 24 * p_tan4)) / p_cos
    
    def zone_number_to_central_longitude(zone_number):
        return (zone_number - 1) * 6 - 180 + 3
    
    return (mathlib.degrees(latitude),
            mathlib.degrees(longitude) + zone_number_to_central_longitude(zone_number))


@vectorize(['float64(float64, float64)'], target='cuda')
def reproj_gpu_lat(x, y):
    lat, lon = to_latlon(x, y, np.int64(31), None, True, None)
    return lat

@vectorize(['float64(float64, float64)'], target='cuda')
def reproj_gpu_lon(x, y):
    lat, lon = to_latlon(x, y, np.int64(31), None, True, None)
    return lon

In [ ]:
import cupy as cp
       
@vectorize(['float32(float32, float32)'], target='cuda')
def reproj_gpu_lat(x, y):
    lat, lon = to_latlon(x, y, cp.int32(31), None, True, None)
    return lat

@vectorize(['float32(float32, float32)'], target='cuda')
def reproj_gpu_lon(x, y):
    lat, lon = to_latlon(x, y, cp.int32(31), None, True, None)
    return lon

class TiffImporter:

    def __init__(self):
        pass

    def latlon_image(self, path):

        # Open input dataset
        indataset = gdal.Open(path, gdal.GA_ReadOnly)
        Projection.setup(indataset.GetProjection())

        # Read geotransform matrix and calculate ground coordinates
        geomatrix = indataset.GetGeoTransform()
        pixel = indataset.RasterXSize
        line = indataset.RasterYSize

        ct = Projection.projector.get_source_to_wgs84_transform()

        latlon_im = cp.zeros([2, line, pixel])
        
        # step 1 - apply geotransform to get the UTM coordinates of the pixel at i,j

        d_i = cp.arange(0, line, 1, dtype=cp.int32).reshape(line, 1)
        d_j = cp.arange(0, pixel, 1, dtype=cp.int32).reshape(1, pixel)
        d_x = cp.empty(shape=(line, pixel), dtype=cp.float32)
        d_y = cp.empty(shape=(line, pixel), dtype=cp.float32)
        
        @vectorize(['float32(int32, int32)'], target='cuda')
        def transform_x_gpu(i, j):
            x = geomatrix[0] + geomatrix[1] * j + geomatrix[2] * i
            x += geomatrix[1] / 2.0
            return x

        @vectorize(['float32(int32, int32)'], target='cuda')
        def transform_y_gpu(i, j):
            y = geomatrix[3] + geomatrix[4] * j + geomatrix[5] * i
            y += geomatrix[5] / 2.0
            return y
        
        transform_x_gpu(d_i, d_j, out=d_x)
        transform_y_gpu(d_i, d_j, out=d_y)
        
        # step 2 - apply reprojection from UTM to WGS84
        d_lat = cp.empty(shape=(line, pixel), dtype=cp.float32)
        d_lon = cp.empty(shape=(line, pixel), dtype=cp.float32)

        reproj_gpu_lat(d_x, d_y, out=d_lat)
        reproj_gpu_lon(d_x, d_y, out=d_lon)
        
        latlon_im[0] = d_lat
        latlon_im[1] = d_lon

        return cp.asnumpy(latlon_im)#latlon_im

In [ ]:
# Making these small changes does simplify things a little, and we also get further speed up. Now down to just over 1 second!

In [ ]:
%%time
importer = TiffImporter()
latlon_im = importer.latlon_image(input_path)